Initial run for Isabel's Data
=================

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
basedir = '../'
sys.path.append(basedir)

import numpy as np
import pandas as pd
import pylab as plt
from IPython.display import display
import math

from lda_for_fragments import Ms2Lda

<h2>Analysis</h2>

<h3>a. Run LDA</h3>

In [2]:
fragment_filename = 'isabelpos_fragments.csv'
neutral_loss_filename = 'isabelpos_losses.csv'
mzdiff_filename = None
ms1_filename = 'isabelpos_ms1.csv'
ms2_filename = 'isabelpos_ms2.csv'
ms2lda = Ms2Lda.lcms_data_from_R(fragment_filename, neutral_loss_filename, mzdiff_filename, 
                             ms1_filename, ms2_filename)

Loading input files
Data shape (435, 1108)


In [3]:
### all the parameters you need to specify to run LDA ###

n_topics = 100
n_samples = 1000
n_burn = 0 # if 0 then we only use the last sample
n_thin = 1 # every n-th sample to use for averaging after burn-in. Ignored if n_burn = 0
alpha = 50.0/n_topics # hyper-parameter for document-topic distributions
beta = 0.1 # hyper-parameter for topic-word distributions

ms2lda.run_lda(n_topics, n_samples, n_burn, n_thin, alpha, beta)

Fitting model...
CGS LDA initialising
............................................
Using Numba for LDA sampling
Preparing words
Preparing Z matrix
DONE
Sample 1   Log likelihood = -1461466.853 
Sample 2   Log likelihood = -1042352.717 
Sample 3   Log likelihood = -728233.192 
Sample 4   Log likelihood = -615263.063 
Sample 5   Log likelihood = -575765.339 
Sample 6   Log likelihood = -556139.565 
Sample 7   Log likelihood = -544430.841 
Sample 8   Log likelihood = -534607.046 
Sample 9   Log likelihood = -528740.628 
Sample 10   Log likelihood = -522917.197 
Sample 11   Log likelihood = -518421.127 
Sample 12   Log likelihood = -515177.828 
Sample 13   Log likelihood = -513795.819 
Sample 14   Log likelihood = -512253.379 
Sample 15   Log likelihood = -509922.274 
Sample 16   Log likelihood = -507959.845 
Sample 17   Log likelihood = -506629.517 
Sample 18   Log likelihood = -505258.369 
Sample 19   Log likelihood = -503907.724 
Sample 20   Log likelihood = -502951.820 
Sample 21   Log

<h3>b. Annotations</h3>

In [4]:
sirius_platform = 'orbitrap'
ms2lda.annotate_with_sirius(sirius_platform, mode='pos', max_ms1=9999) # mode is either 'pos' or 'neg'

Running SIRIUS annotation with parameters:
- platform = orbitrap
- mode = pos
- ppm_max = 5
- min_score = 0.01
- max_ms1 = 9999

    1/  435 pID  452 m/z 385.18096 int 8.1252e+05 n_frags  5	ACCEPT	3 fragment(s) annotated with score 12.63
    2/  435 pID  631 m/z 513.25313 int 8.3645e+05 n_frags  1	ACCEPT	0 fragment(s) annotated with score 0.20
    3/  435 pID  672 m/z 358.25877 int 6.2059e+05 n_frags  9	ACCEPT	6 fragment(s) annotated with score 21.38
    4/  435 pID  693 m/z 391.25098 int 7.0632e+05 n_frags 13	ACCEPT	10 fragment(s) annotated with score 21.22
    5/  435 pID  701 m/z 763.38784 int 3.0677e+06 n_frags  5	ACCEPT	4 fragment(s) annotated with score 1.63
    6/  435 pID  716 m/z 742.45888 int 1.7964e+06 n_frags  7	ACCEPT	5 fragment(s) annotated with score 9.68
    7/  435 pID  718 m/z 880.59025 int 8.6466e+05 n_frags  5	ACCEPT	5 fragment(s) annotated with score 2.88
    8/  435 pID  723 m/z 312.21684 int 7.1932e+05 n_frags 16	ACCEPT	14 fragment(s) annotated with score 50.33
 

In [16]:
ms2lda = Ms2Lda.resume_from('isabel.project')

Project loaded from isabel.project time taken = 5.55397295952
 - input_filenames = 
	isabelpos_fragments.csv
	isabelpos_losses.csv
	isabelpos_ms1.csv
	isabelpos_ms2.csv
 - df.shape = (435, 1108)
 - K = 100
 - alpha = 0.5
 - beta = 0.1
 - number of samples stored = 1
 - last_saved_timestamp = Thu Oct  8 18:19:16 2015
 - message = Initial run on Isabel data by Joe


In [18]:
proton_mass = 1.00727645199076

# peaklist_filename = 'peaklist_POS.csv'
peaklist_filename = 'sanitized_ideom_output.csv'
peaklist = pd.read_csv(peaklist_filename)
annot = peaklist[['mz', 'rt', 'int', 'metabolite','AN3057']]
# annot = annot[pd.notnull(annot['Metabolite'])] # drop NaN metabolites


label_len = 20
mass_tol = 10
rt_tol = 120

# for each row in the IDIOM annotation results 
total_match = 0
for row in annot.itertuples():
    
    # get mz, rt, intensity and the putative metabolite identity
    mz = row[1] + proton_mass
    rt = row[2]
    intensity = row[3]
    label = row[4]
    fold_change = row[5]
    print fold_change
    
    # truncate long metabolite names
#     label = (label[:label_len] + '..') if len(label) > label_len else label    
    print "Processing " + str((mz, rt, intensity)) + " " + str(label)

    # find matching MS1 within mass_tol and rt_tol
    interval = mz*mass_tol*1e-6
    mass_start, mass_end = (mz-interval, mz+interval)
    rt_start, rt_end = (rt-rt_tol, rt+rt_tol)
    
    match_mass = (ms2lda.ms1.mz>mass_start) & (ms2lda.ms1.mz<mass_end)
    match_rt = (ms2lda.ms1.rt>rt_start) & (ms2lda.ms1.rt<rt_end)
    match = match_mass & match_rt
    
    res = ms2lda.ms1[match]
    if len(res) == 1:

        # if only 1 match, immediately annotate
        print "Found 1 hit"
        idx = res.index[0]
        ms2lda.ms1.loc[[idx], 'annotation'] = label
        ms2lda.ms1.loc[[idx], 'fold_change'] = fold_change
        display(res)
        total_match += 1

    elif len(res) > 0:

        print "Found %d hits" % len(res)        
        
        # if multiple matches, find the closest in RT   
        closest = None
        min_dist = sys.maxint
        for match_res in res.itertuples():
            match_rt = match_res[4]
            match_mz = match_res[5]
            dist = math.sqrt((match_rt-rt)**2 + (match_mz-mz)**2)
            if dist < min_dist:
                min_dist = dist
                closest = match_res

        idx = closest[0]
        ms2lda.ms1.loc[[idx], 'annotation'] = label
        ms2lda.ms1.loc[[idx], 'fold_change'] = fold_change
        display(res)
        total_match += 1        
        
    print
    
print "total_match=" + str(total_match) + "/" + str(annot.shape[0])

1.482959336
Processing (71.085536421990767, 232.57195980000003, 34996) nan

0.863648405
Processing (74.023649131990766, 695.99287670000012, 3532) Iminoglycine

1.668358028
Processing (74.060028581990764, 595.91434649999997, 48715) Aminoacetone
Found 2 hits


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
3888,3888,0,1,508.189,74.060018,1482000.625,1,20,0,C3H7NO,NaN
5502,5502,0,1,615.361,74.060035,4867594.000,1,0,0,Aminoacetone,1.668358



0.863951282
Processing (76.039059521990765, 1356.103124, 29573) nan

0.812166902
Processing (76.039295691990759, 822.14852840000003, 245819) Glycine

1.152930833
Processing (76.075680391990758, 648.14350860000002, 57397) (R)-1-Aminopropan-2-ol

1.012631981
Processing (77.008135031990761, 1380.0902100000001, 22659) nan

0.891934301
Processing (77.023065081990765, 410.51318800000001, 757550) nan

0.942447857
Processing (77.649609521990769, 257.633667, 2485) nan

3.180295842
Processing (78.099681331990766, 256.86133239999998, 6149) nan

0.926261816
Processing (78.211064611990764, 253.77637510000002, 14225) nan

0.994963144
Processing (78.984402421990765, 1287.2287550000001, 8819) nan

0.755687688
Processing (79.021203161990769, 408.629795, 130333) nan

0.812627845
Processing (79.021206391990759, 5.9989650879999994, 109253) nan

0.947069883
Processing (80.933275081990757, 717.23124689999997, 42302) Br-

0.499881122
Processing (80.947727751990755, 721.29287470000008, 16973) nan

0.77309146

,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
3606,3606,0,1,536.499,86.096339,15411655,1,0,0,C5H11N,NaN
3607,3607,0,1,521.822,86.096340,11351523,1,77,0,NaN,0.879492



0.597283755
Processing (87.043988721990758, 655.28750100000002, 4522) 2-Butenoate

1.122427753
Processing (87.044030321990761, 794.88474659999997, 56969) Diacetyl

1.018365016
Processing (87.091696071990768, 1231.804695, 166322) nan

1.418065743
Processing (87.10241521199076, 644.7585004, 3004) nan

1.023331088
Processing (88.039292101990767, 801.01649860000009, 15314) 2-Aminoacrylate

0.516191923
Processing (89.023254561990768, 462.82610319999998, 2810079) Pyruvate

1.068447619
Processing (89.023315021990769, 784.43687179999995, 352474) 3-Oxopropanoate

1.031052712
Processing (90.018562751990757, 729.32312390000004, 13831) Oxamate

0.649730571
Processing (90.054894181990761, 791.51574579999999, 10722286) L-Alanine

0.495358583
Processing (90.054958671990761, 1336.0123390000001, 391083) Sarcosine

0.31837299
Processing (90.054959461990762, 1162.7030600000001, 185944) Sarcosine

1.095888835
Processing (90.054974261990765, 1243.8015809999999, 450100) D-Alanine

0.885172658
Processing (9

...


1.534093429
Processing (834.62329785199074, 241.6501083, 14244) PS(17:0/22:0)

0.688329983
Processing (836.54240585199068, 214.16510449999998, 30969) PS(40:6)

0.614805455
Processing (838.55784855199067, 213.59152090000003, 21442) PS(40:5)

0.608501415
Processing (838.62959105199081, 237.905688, 313188) PC(40:4)

0.548972246
Processing (840.57403615199087, 212.33837450000001, 40241) PS(40:4)

0.466345636
Processing (842.58937335199073, 212.09633259999998, 46975) PS(40:3)

0.419243637
Processing (844.60523695199072, 212.45361589999999, 38908) PS(40:2)

0.664692336
Processing (851.6690241519907, 912.05187609999996, 5946) nan

0.774350064
Processing (858.59831385199072, 237.50433380000001, 145153) PC(42:8)

0.644162562
Processing (868.57211105199076, 240.94289320000001, 19908) nan

1.149888376
Processing (877.26287545199079, 200.86426530000003, 1773) nan

1.049279151
Processing (880.58320985199066, 236.70545959999998, 163245) PC(44:11)
Found 1 hit


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
718,718,0,1,205.485,880.590255,864663.9375,1,0,0,"[PI (18:0/18:0)] 1,2-di-(9Z-octadecenoyl)-sn-g...",0.649157



0.649156532
Processing (880.59001535199081, 219.55060449999999, 94209) [PI (18:0/18:0)] 1,2-di-(9Z-octadecenoyl)-sn-glycero-3-phospho-(1'-myo-inositol)(ammonium salt)
Found 1 hit


,peakID,MSnParentPeakID,msLevel,rt,mz,intensity,Sample,GroupPeakMSn,CollisionEnergy,annotation,fold_change
718,718,0,1,205.485,880.590255,864663.9375,1,0,0,PC(44:11),1.049279



0.814858263
Processing (882.59855175199061, 236.7771889, 87839) PC(44:10)

1.79991236
Processing (892.57175865199076, 240.29648950000001, 10169) nan

0.933481385
Processing (975.38491285199075, 587.93751529999997, 128634) nan

total_match=97/1154


<h3>c. Save Project</h3>

In [7]:
# leave the message parameter out if nothing to say
ms2lda.save_project('isabel.project', message="Initial run on Isabel data by Joe")

Project saved to isabel.project time taken = 2.27142715454


<hr/>

<h2>Results</h2>

In [8]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
basedir = '../'
sys.path.append(basedir)

from lda_for_fragments import Ms2Lda

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
ms2lda = Ms2Lda.resume_from('isabel.project')

Project loaded from isabel.project time taken = 5.01552891731
 - input_filenames = 
	isabelpos_fragments.csv
	isabelpos_losses.csv
	isabelpos_ms1.csv
	isabelpos_ms2.csv
 - df.shape = (435, 1108)
 - K = 100
 - alpha = 0.5
 - beta = 0.1
 - number of samples stored = 1
 - last_saved_timestamp = Thu Oct  8 18:19:16 2015
 - message = Initial run on Isabel data by Joe


In [46]:
# Thresholding the doc_topic and topic_word matrices
ms2lda.do_thresholding(th_doc_topic=0.05, th_topic_word=0.01)

In [47]:
# ms2lda.print_topic_words()

In [48]:
special_nodes = []
for i in ms2lda.ms1.index:
    fold_change = ms2lda.ms1.loc[i,'fold_change']
    if fold_change > 1.5:
        print i
        special_nodes.append(("doc_{}".format(),'#CC0000'))

745
1730
1848
1855
1868
1951
2018
2998
5489
5502


In [49]:
interactive=True
consistency=0.0
# special_nodes = None
ms2lda.plot_lda_fragments(consistency=consistency, interactive=interactive, to_highlight=special_nodes)

Ranking topics ...
 - topic 0 h-index = 2
 - topic 1 h-index = 5
 - topic 2 h-index = 1
 - topic 3 h-index = 4
 - topic 4 h-index = 2
 - topic 5 h-index = 2
 - topic 6 h-index = 3
 - topic 7 h-index = 3
 - topic 8 h-index = 1
 - topic 9 h-index = 2
 - topic 10 h-index = 2
 - topic 11 h-index = 2
 - topic 12 h-index = 1
 - topic 13 h-index = 4
 - topic 14 h-index = 7
 - topic 15 h-index = 1
 - topic 16 h-index = 2
 - topic 17 h-index = 3
 - topic 18 h-index = 3
 - topic 19 h-index = 5
 - topic 20 h-index = 2
 - topic 21 h-index = 4
 - topic 22 h-index = 3
 - topic 23 h-index = 2
 - topic 24 h-index = 2
 - topic 25 h-index = 2
 - topic 26 h-index = 2
 - topic 27 h-index = 2
 - topic 28 h-index = 2
 - topic 29 h-index = 7
 - topic 30 h-index = 2
 - topic 31 h-index = 4
 - topic 32 h-index = 5
 - topic 33 h-index = 3
 - topic 34 h-index = 1
 - topic 35 h-index = 2
 - topic 36 h-index = 2
 - topic 37 h-index = 3
 - topic 38 h-index = 3
 - topic 39 h-index = 7
 - topic 40 h-index = 2
 - topi

127.0.0.1 - - [12/Oct/2015 12:03:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:33] "GET /LDAvis.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:33] "GET /d3.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:33] "GET /LDAvis.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:33] "GET /images/graph_example.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:33] "GET /images/default_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2015 12:03:37] "GET /graph.html?degree=0&visID=ldavis_el917944473653362922967391 HTTP/1.1" 200 -


Serving dynamic json file -- threshold = 0
to_highlight = [745, 1730, 1848, 1855, 1868, 1951, 2018, 2998, 5489, 5502]
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 62273)
----------------------------------------
Serving dynamic json file -- threshold = 0
to_highlight = [745, 1730, 1848, 1855, 1868, 1951, 2018, 2998, 5489, 5502]
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 62274)
----------------------------------------
Serving dynamic json file -- threshold = 0

Traceback (most recent call last):
  File "//anaconda/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "//anaconda/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "//anaconda/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "//anaconda/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "//anaconda/lib/python2.7/BaseHTTPServer.py", line 340, in handle
    self.handle_one_request()
  File "//anaconda/lib/python2.7/BaseHTTPServer.py", line 328, in handle_one_request
    method()
  File "../visualisation/pyLDAvis/_server.py", line 141, in do_GET
    GlobalVariable.degree)
  File "../visualisation/networkx/lda_visualisation.py", line 116, in get_json_from_docdf
    to_highlight_labels = [x[0] for x in to_highlight]
IndexError: invalid index to


to_highlight = [745, 1730, 1848, 1855, 1868, 1951, 2018, 2998, 5489, 5502]
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 62275)
----------------------------------------

stopping Server...


  File "//anaconda/lib/python2.7/SocketServer.py", line 295, in _handle_request_noblock
    self.process_request(request, client_address)
  File "//anaconda/lib/python2.7/SocketServer.py", line 321, in process_request
    self.finish_request(request, client_address)
  File "//anaconda/lib/python2.7/SocketServer.py", line 334, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "//anaconda/lib/python2.7/SocketServer.py", line 655, in __init__
    self.handle()
  File "//anaconda/lib/python2.7/BaseHTTPServer.py", line 340, in handle
    self.handle_one_request()
  File "//anaconda/lib/python2.7/BaseHTTPServer.py", line 328, in handle_one_request
    method()
  File "../visualisation/pyLDAvis/_server.py", line 141, in do_GET
    GlobalVariable.degree)
  File "../visualisation/networkx/lda_visualisation.py", line 116, in get_json_from_docdf
    to_highlight_labels = [x[0] for x in to_highlight]
IndexError: invalid index to scalar variable.
